In [ ]:
pip install datasets transformers soundfile librosa evaluate jiwer

# PARTE 1: IMPORT e CARICAMENTO del dataset LibriSpeech

In [ ]:
from datasets import load_dataset
import torchaudio
import torchaudio.transforms as T
import torch
import torch.nn as nn
from torch.nn import CTCLoss
from torch.utils.data import DataLoader
from jiwer import wer, cer
import time
from torch.utils.data import Subset
import random

train_dataset = load_dataset("./local_dataset_loader", name="local_dataset_loader" ,split="train", data_dir="./libriSpeech_data/LibriSpeech", trust_remote_code=True)
eval_dataset = load_dataset("./local_dataset_loader", name="local_dataset_loader", split="validation", data_dir="./libriSpeech_data/LibriSpeech", trust_remote_code=True)
test_dataset = load_dataset("./local_dataset_loader", name="local_dataset_loader",split="test", data_dir="./libriSpeech_data/LibriSpeech", trust_remote_code=True)
print("Caricamento dataset LibriSpeech avvenuto!")

Caricamento dataset LibriSpeech avvenuto!


# PARTE 2: VOCABOLARIO

Serve a mappare ogni carattere (o token) a un indice numerico. Nel nostro caso, il modello trascrive caratteri (non parole), quindi il vocabolario è un char-level mapping 
{"'": 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, '|': 27, '<blank>': 28, '<pad>': 29, '<unk>': 30}

In [ ]:
def build_vocab_dict(dataset):
    all_text = " ".join(dataset["text"])
    unique_chars = sorted(set(all_text) - {" "})
    unique_chars.append("|")

    vocab_dict = {c: i for i, c in enumerate(unique_chars)}

    vocab_dict["<blank>"] = len(vocab_dict)
    vocab_dict["<pad>"] = len(vocab_dict) 
    vocab_dict["<unk>"] = len(vocab_dict)

    return vocab_dict

vocab_dict = build_vocab_dict(train_dataset)
print(vocab_dict)

{"'": 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, '|': 27, '<blank>': 28, '<pad>': 29, '<unk>': 30}


# PARTE 3: PRE_PROCESSING

# Conversione dell’audio in tensori numerici
I modelli deep learning non possono lavorare direttamente con file audio o oggetti “audio”.
Il preprocessing estrae l’audio in forma di tensore float32, lo normalizza, e (se necessario) lo ricampiona alla frequenza standard (16kHz).

# Tokenizzazione del testo
Trasforma la trascrizione da testo in una sequenza di interi (labels), dove ogni carattere è mappato a un ID del vocabolario.
È essenziale per usare la CTC Loss, che lavora su sequenze di ID e non stringhe di testo.

# Dopo questo preprocessing abbiamo: 
"input_values" → tensore audio normalizzato, pronto per essere passato a CNN o Transformer.
"labels" → lista di ID dei caratteri della trascrizione, per calcolare la CTC loss.

# Fare il mapping su train, validation e test ha senso: 
train: per addestrare il modello con input numerici
validation: per monitorare la performance durante il training
test: per valutare il modello finalizzato

In [ ]:

class SimpleProcessor:
    def __init__(self, vocab_dict, target_sampling_rate=16000, augment=True):
        self.vocab = vocab_dict
        self.target_sr = target_sampling_rate
        self.augment = augment

    def preprocess_audio(self, audio_array, orig_sr):
        if orig_sr != self.target_sr:
            resampler = torchaudio.transforms.Resample(orig_sr, self.target_sr)
            audio_array = resampler(torch.tensor(audio_array).float())
        else:
            audio_array = torch.tensor(audio_array).float()

        if self.augment:
            if random.random() < 0.5:
                volume_factor = random.uniform(0.8, 1.2)
                audio_array = audio_array * volume_factor

            if random.random() < 0.3:
                noise_factor = random.uniform(0.001, 0.01)
                noise = torch.randn_like(audio_array) * noise_factor
                audio_array = audio_array + noise

        audio_array = (audio_array - audio_array.mean()) / (audio_array.std() + 1e-5)

        return audio_array

    def tokenize_text(self, text):
        text = text.replace(" ", "|")
        return [self.vocab.get(c, self.vocab["<unk>"]) for c in text]

    def __call__(self, audio, sampling_rate, text=None):
        inputs = self.preprocess_audio(audio, sampling_rate)

        result = {"input_values": inputs}
        if text is not None:
            result["labels"] = torch.tensor(self.tokenize_text(text), dtype=torch.long)
        return result

processor = SimpleProcessor(vocab_dict)

def preprocess(batch):
    audio = batch["audio"]["array"]
    sr = batch["audio"]["sampling_rate"]
    text = batch["text"]

    processed = processor(audio, sampling_rate=sr, text=text)
    return {
        "input_values": processed["input_values"],
        "labels": processed["labels"]
    }
    
print("Avvio preprocessing ottimizzato..")

train_dataset_processed = train_dataset.map(
    preprocess,
    remove_columns=train_dataset.column_names,
    num_proc=4,
)

eval_dataset_processed = eval_dataset.map(
    preprocess,
    remove_columns=eval_dataset.column_names,
    num_proc=4,
)

test_dataset_processed = test_dataset.map(
    preprocess,
    remove_columns=test_dataset.column_names,
    num_proc=4,
)

print("Fine preprocessing!")

Avvio preprocessing ottimizzato..


Map (num_proc=4):   0%|          | 0/28539 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2703 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2620 [00:00<?, ? examples/s]

Fine preprocessing!


In [ ]:
#esempio per mostrare il preProcessing
print(train_dataset_processed[0].keys())
print(train_dataset_processed[0]["input_values"][:20])
print(train_dataset_processed[0]["labels"][:20])

print(eval_dataset_processed[0]["input_values"][:20])
print(eval_dataset_processed[0]["labels"][:20])

print(test_dataset_processed[0]["input_values"][:20])
print(test_dataset_processed[0]["labels"][:20])

# In base all'output accade che:

# vocabolario: {"'": 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 
#               'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, '|': 27, '<pad>': 28, 
#               '<unk>': 29}

# primi 20 caratteri del primo elemento: H A D   L A I D   B E F O R E
# diventano: [H, A, D, '|', L, A, I, D, '|', B, E, F, O, R, E] -> [8, 1, 4, 27, 12, 1, 9, 4, 27, 2, 5, 6, 15, 18, 5] 
# label: [8, 1, 4, 27, 12, 1, 9, 4, 27, 2, 5, 6, 15, 18, 5, 27, ...]

#lo stesso per l'eval e test

dict_keys(['input_values', 'labels'])
[-0.15206488966941833, -0.17493396997451782, 0.1320481300354004, 0.003173402277752757, -0.020163103938102722, 0.029104067012667656, -0.07104626297950745, 0.08140091598033905, -0.09795332700014114, 0.039900705218315125, -0.07659906148910522, 0.14178574085235596, -0.014021406881511211, 0.0908619835972786, -0.2531350553035736, 0.013963003642857075, -0.2295738309621811, 0.06235656514763832, 0.053652241826057434, 0.14166110754013062]
[8, 1, 4, 27, 12, 1, 9, 4, 27, 2, 5, 6, 15, 18, 5, 27, 8, 5, 18, 27]
[0.0007413655985146761, 0.0014564606826752424, 0.0021715557668358088, 0.0007413655985146761, 0.0007413655985146761, 0.0021715557668358088, 0.0021715557668358088, 0.0014564606826752424, 0.0014564606826752424, 0.0014564606826752424, 0.0014564606826752424, 0.0014564606826752424, 0.0014564606826752424, 0.0014564606826752424, 0.002886650850996375, 0.0021715557668358088, 0.0021715557668358088, 0.0021715557668358088, 0.0014564606826752424, 0.0014564606826752424]


# PARTE 4: MODELLO

1. Prende l’audio grezzo (onda sonora) e lo trasforma in un’immagine chiamata spettrogramma Mel-log - una rappresentazione che mostra come l’energia del suono cambia nel tempo e nelle frequenze, più facile da interpretare per la rete.

2. Passa questo spettrogramma a una CNN (convoluzione) che estrae caratteristiche importanti e riduce la lunghezza della sequenza nel tempo riassumendo l’informazione utile.
 
3. Le caratteristiche ottenute vengono date a un Transformer, che capisce le dipendenze temporali più complesse e le relazioni  tra le parti della sequenza audio

4. Infine, un layer lineare trasforma queste informazioni in punteggi (logits) per ogni possibile carattere del vocabolario, cioè decide quali lettere o simboli sono più probabili in ogni momento.
Questi punteggi vengono poi usati per calcolare la perdita (loss) con la CTC e per generare il testo trascritto.

IN GENERALE: è utile perché trasforma il suono grezzo, che è difficile da interpretare direttamente, in una rappresentazione più ricca e strutturata (lo spettrogramma Mel-log) che cattura le caratteristiche importanti del parlato.


In [ ]:

class SimpleASRModel(nn.Module):
    def __init__(self, vocab_size):
        super(SimpleASRModel, self).__init__()

        # 1. Trasformazione audio -> log-Mel spectrogram
        self.melspec = T.MelSpectrogram(
            sample_rate=16000,
            n_fft=400,
            hop_length=160,
            n_mels=128
        )
        self.log_transform = lambda x: torch.log(x + 1e-5)

        # 2. CNN per feature extraction (2 layer)
        self.cnn = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=5, stride=2, padding=2),
            nn.ReLU()
        )

        # 3. Transformer Encoder
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=256,
                nhead=4,
                dim_feedforward=512
            ),
            num_layers=3
        )

        self.classifier = nn.Linear(256, vocab_size)

    def forward(self, x):
        """
        x: (batch, audio_len) - waveform normalizzato
        """
        x = self.melspec(x)
        x = self.log_transform(x)

        x = self.cnn(x)
        x = x.permute(2, 0, 1)

        x = self.transformer(x)

        x = self.classifier(x)

        return x

# PARTE 5: COLLATE

Quando si usa un DataLoader con un batch di dati di lunghezze variabili (come onde audio o sequenze di caratteri), serve una funzione collate per allineare tutto correttamente in un batch tensoriale. 

Quindi questo prepara il batch per il training, gestendo sequenze variabili.

cosa succede:
1. vengono estratte delle sequenze audio (input_values) e testo (labels):
2. viene applicato del padding alle sequenze audio (utile nel caso della eval)
3. vengono calcolate le lunghezze originali delle sequenze audio prima del padding
4. viene applicato il padding alle label testuali
5. vengono calcolate le lunghezze reali delle label
6. viene ritornato tutto in un unico dizionario

In [ ]:
def collate(batch):
    
    inputs = [torch.tensor(item["input_values"]) for item in batch]
    targets = [torch.tensor(item["labels"]) for item in batch]  
    
    inputs_padded = nn.utils.rnn.pad_sequence(inputs, batch_first=True)
    
    input_lengths = torch.tensor([len(i) for i in inputs])

    targets_padded = nn.utils.rnn.pad_sequence(targets, batch_first=True, padding_value=vocab_dict["<pad>"])

    target_lengths = torch.tensor([len(t) for t in targets])
        
    return {
        "input_values": inputs_padded, 
        "labels": targets_padded, 
        "input_lengths": input_lengths, 
        "label_lengths": target_lengths
    }

# PARTE 5: TRAIN LOOP



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleASRModel(vocab_size=len(vocab_dict)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# CTC usa il token blank per gestire l'allineamento temporale
criterion = CTCLoss(blank=vocab_dict["<blank>"], zero_infinity=True)

subset_indices = list(range(1000))
small_dataset = Subset(train_dataset_processed, subset_indices)

#train_loader = DataLoader(train_dataset_processed, batch_size=4, shuffle=True, collate_fn=collate)

train_loader = DataLoader(small_dataset, batch_size=4, shuffle=True, collate_fn=collate)
dev_loader = DataLoader(eval_dataset_processed, batch_size=4, shuffle=False, collate_fn=collate)

print("Numero di batch per epoca:", len(train_loader))

# La funzione greedy_decode(log_probs, vocab_dict) serve per convertire le probabilità logaritmiche (output del modello) 
# in una trascrizione testuale leggibile, scegliendo per ogni time step il carattere più probabile.
def greedy_decode(log_probs, vocab_dict):
    inv_vocab = {v: k for k, v in vocab_dict.items()}
    pred_ids = torch.argmax(log_probs, dim=-1)
    pred_texts = []

    for b in range(pred_ids.shape[1]):
        prev = None
        sentence = []
        for t in range(pred_ids.shape[0]):
            idx = pred_ids[t, b].item()
            if idx != prev and idx != vocab_dict["<blank>"] and idx != vocab_dict["<pad>"]:
                if idx in inv_vocab:
                    sentence.append(inv_vocab[idx])
                prev = idx
        pred_texts.append("".join(sentence))
    return pred_texts

# Con stride=2, ogni layer dimezza approssimativamente la dimensione temporale. Quindi:
#    Dopo prima conv: T → T/2
#    Dopo seconda conv: T/2 → T/4
# Ecco perché compute_output_lengths divide per 2 due volte.
def compute_output_lengths(input_lengths, num_layers=2, stride=2):
    for _ in range(num_layers):
        input_lengths = (input_lengths + 1) // stride
    return input_lengths

for epoch in range(20):
    model.train()
    total_loss = 0
    start_time = time.time()

    for batch in train_loader:
        inputs = batch["input_values"].to(device)
        targets = batch["labels"].to(device)
        input_lengths = batch["input_lengths"].to(device)
        target_lengths = batch["label_lengths"].to(device)

        # Ecco cosa avveine:
        # 1. Abbiamo un audio (waveform) lungo input_lengths (in campioni, es. 16000 = 1 secondo).
        # 2. Lo trasformiamo in un mel-spectrogramma, cioè una sequenza di “frame” nel tempo.
        # 3. Passiamo quei frame nella CNN, che ha 2 layer con stride=2. La lunghezza la calcoliamo con compute
        # 4. La CTC Loss ha bisogno di sapere:
        #    T: quante previsioni temporali ha il modello (adjusted_input_lengths)
        #    U: quanto è lunga la trascrizione (target_lengths)

        #Waveform -> Mel-Spectrogram
        # mel_frame_length = ((input_len - n_fft) // hop_length) + 1
        mel_frame_length = ((input_lengths - 400) // 160) + 1
        #La CNN ha 2 layer con stride=2, quindi la dimensione temporale si riduce
        # È la vera lunghezza temporale dell’output del modello, cioè quante “previsioni” fa il modello nel tempo per ogni audio nel batch.
        adjusted_input_lengths = compute_output_lengths(mel_frame_length).to(device)

        outputs = model(inputs)
        log_probs = nn.functional.log_softmax(outputs, dim=-1)

        # Appiattisci le label reali rimuovendo i padding
        flattened_targets = torch.cat([targets[i, :target_lengths[i]] for i in range(targets.size(0))])

        loss = criterion(log_probs, flattened_targets, adjusted_input_lengths, target_lengths)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()        

    duration = time.time() - start_time
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}, Durata: {duration:.2f}s")

    model.eval()
    all_preds, all_targets = [], []

    # ======= VALUTAZIONE SU DEV ========

    with torch.no_grad():
        for batch in dev_loader:
            inputs = batch["input_values"].to(device)
            targets = batch["labels"]
            target_lengths = batch["label_lengths"]

            outputs = model(inputs)
            log_probs = nn.functional.log_softmax(outputs, dim=-1)

            pred_texts = greedy_decode(log_probs.cpu(), vocab_dict)

            inv_vocab = {v: k for k, v in vocab_dict.items()}
            for i, length in enumerate(target_lengths):
                target_ids = targets[i][:length].tolist()
                target_text = "".join([inv_vocab[id] for id in target_ids])
                target_text = target_text.replace("|", " ")
                all_targets.append(target_text)

            all_preds.extend(pred_texts)

    wer_score = wer(all_targets, all_preds)
    cer_score = cer(all_targets, all_preds)
    print(f"Epoch {epoch+1} - WER: {wer_score:.4f}, CER: {cer_score:.4f}")

    print("\nEsempi di predizioni:")
    for i in range(min(3, len(all_preds))):
        print(f"Ref: {all_targets[i]}")
        print(f"Pred: {all_preds[i]}")
    print("-" * 50)

Numero di batch per epoca: 250
Epoch 1, Loss: 2.8777, Durata: 91.18s
Epoch 1 - WER: 1.0000, CER: 0.9497

Esempi di predizioni:
Ref: SHORTLY AFTER PASSING ONE OF THESE CHAPELS WE CAME SUDDENLY UPON A VILLAGE WHICH STARTED UP OUT OF THE MIST AND I WAS ALARMED LEST I SHOULD BE MADE AN OBJECT OF CURIOSITY OR DISLIKE
Pred: |A|A|T|A|IN|S|AN|A|A|A|E|E|A|R|A|E|N|A|R|
Ref: MY GUIDES HOWEVER WERE WELL KNOWN AND THE NATURAL POLITENESS OF THE PEOPLE PREVENTED THEM FROM PUTTING ME TO ANY INCONVENIENCE BUT THEY COULD NOT HELP EYEING ME NOR I THEM
Pred: |A|A|AT|R|R|R|AE|N|A|A|A|R|E|T|R|RE|R
Ref: THE STREETS WERE NARROW AND UNPAVED BUT VERY FAIRLY CLEAN
Pred: I|AR|R|A|
--------------------------------------------------
Epoch 2, Loss: 2.4514, Durata: 112.85s
Epoch 2 - WER: 1.0000, CER: 0.7488

Esempi di predizioni:
Ref: SHORTLY AFTER PASSING ONE OF THESE CHAPELS WE CAME SUDDENLY UPON A VILLAGE WHICH STARTED UP OUT OF THE MIST AND I WAS ALARMED LEST I SHOULD BE MADE AN OBJECT OF CURIOSITY OR DISLIKE
Pre